# Segment Anything Model for Geospatial Data

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb)

This notebook shows how to use segment satellite imagery using the Segment Anything Model (SAM) with a few lines of code.

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.

## Install dependencies

Uncomment and run the following cell to install the required dependencies.


In [14]:
%pip install "segment-geospatial[samgeo3]"

## Import libraries

In [15]:
import leafmap
from samgeo import SamGeo3, download_file

## Create an interactive map

In [6]:
url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/uc_berkeley.tif"
image_path = download_file(url)

m = leafmap.Map()
m.add_raster(image_path,layer_name="Satellite image")
m

uc_berkeley.tif already exists. Skip downloading. Set overwrite=True to overwrite.


Map(center=[37.87135, -122.25784999999999], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [ ]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-95.3704, 29.6762, -95.368, 29.6775]

## Download map tiles

Download maps tiles and mosaic them into a single GeoTIFF file

In [ ]:
image = "satellite.tif"

Besides the `satellite` basemap, you can use any of the following basemaps returned by the `get_basemaps()` function:

In [ ]:
# get_basemaps().keys()

Specify the basemap as the source.

In [ ]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [ ]:
m.layers[-1].visible = False  # turn off the basemap
m.add_raster(image, layer_name="Image")
m

![](https://i.imgur.com/KAm84IY.png)

## **Request access to SAM3**

In [16]:
from huggingface_hub import login
login()

## Initialize SAM class

In [18]:
sam = SamGeo3(
    backend= "transformers",
    device=None,
    checkpoint_path=None,
    load_from_HF=True,
)

Using cuda device and transformers backend


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/facebook/sam3.
403 Client Error. (Request ID: Root=1-6959570a-3cb2bbcf4ca5d8ad635571c1;214ef8ad-c3c5-4787-9870-85b0bb5039c5)

Cannot access gated repo for url https://huggingface.co/facebook/sam3/resolve/main/config.json.
Access to model facebook/sam3 is restricted and you are not in the authorized list. Visit https://huggingface.co/facebook/sam3 to ask for access.

## Segment the image

Set `batch=True` to segment the image in batches. This is useful for large images that cannot fit in memory.

In [ ]:
mask = "segment.tif"
sam.generate(
    image, mask, batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255
)

## Polygonize the raster data

Save the segmentation results as a GeoPackage file.

In [ ]:
vector = "segment.gpkg"
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

You can also save the segmentation results as any vector data format supported by GeoPandas.

In [ ]:
shapefile = "segment.shp"
sam.tiff_to_vector(mask, shapefile)

## Visualize the results

In [ ]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector(vector, layer_name="Vector", style=style)
m

![](https://i.imgur.com/Ysq3u7E.png)